# Data Visualisations with statistical tests

# Objectives

This notebook aims to visually explore and statistically test the hypotheses that have been outlined in my readme file.

1. Analysing the impact of discounts on profitability: I aim to determine whether or not higher discount rates lead to reduced profits.
2. I aim to evaluate the profitability of different product categories by identifying which categories and sub-categories generate the most profit.
3. Determine whether certain shipping modes or regions result in longer delivery times.
4. Examine whether larger orders lead to higher profits.

# Preparing the data for visualisation

In [9]:
import os

# Define the correct project root
project_root = "C:\\Users\\conor\\Desktop\\DA course\\SuperstoreSales"

# Move up only if currently in the 'jupyter_notebooks' folder
if "jupyter_notebooks" in os.getcwd():
    os.chdir(project_root)
    print(f"Changed working directory to: {os.getcwd()}")
else:
    print(f"Already in the correct directory: {os.getcwd()}")

Already in the correct directory: C:\Users\conor\Desktop\DA course\SuperstoreSales


In [10]:
# Import libraries that will be used throughout the notebook
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Load cleaned dataset and ensure it is displaying correctly
df = pd.read_csv('C:\\Users\\conor\\Desktop\\DA course\\SuperstoreSales\\Cleaned_data\\superstore_cleaned.csv')
df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Segment,City,State,Region,Product ID,Category,Sub-Category,Sales,Quantity,Discount,Profit,Delivery Time,Profit Margin,Total Discount Effect,Order Month
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,Henderson,Kentucky,South,FUR-BO-10001798,Furniture,Bookcases,261.9600,2,0.00,41.913600,3,0.160000,0.00,2016-11
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,Consumer,Henderson,Kentucky,South,FUR-CH-10000454,Furniture,Chairs,731.9400,3,0.00,219.582000,3,0.300000,0.00,2016-11
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,Corporate,Los Angeles,California,West,OFF-LA-10000240,Office Supplies,Labels,14.6200,2,0.00,6.871400,4,0.470000,0.00,2016-06
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,South,FUR-TA-10000577,Furniture,Tables,957.5775,5,0.45,-319.264953,7,-0.333409,2.25,2015-10
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,South,OFF-ST-10000760,Office Supplies,Storage,22.3680,2,0.20,2.516400,7,0.112500,0.40,2015-10


# Section 1

Hyposthesis: Higher discount levels impact profitability negatively.

Null Hyposthesis: There is no relationship between discount levels and profits.

In [11]:
# had to manually install nbformat to fix error message, only worked when running it here and not through requirements file or terminal
!pip install nbformat --upgrade

In [12]:
# Scatter plot showing Discount vs Profit

fig = px.scatter(df, x='Discount', y='Profit', 
                 color='Category', # Color by category to show which products are most affected by discount
                 title='Discount vs Profit', 
                 labels={'Discount': 'Discount (Fractional)', 'Profit': 'Profit ($)'})
fig.show()


This scatter plot clearly shows the relationship between higher discount rates and profits earned, every single item that was sold at a discount of greater than 45% was sold at a loss.

In [13]:
# Create the boxplot in Plotly
fig = px.box(df, x="Discount", y="Profit",
             title="Profit Distribution Across Discount Levels",
             labels={"Discount ": "Discount(Fractional)", "Profit": "Profit ($)"},
             color="Discount")  # Different colors for each different discount rate

fig.show()

With the boxplots there is also a clear, albeit not perfect, downtrend when comparing profit with discount as the discount value increases.

# Statistical test to prove/disprove hypothesis

I looked through our course content to try and decide which statistical test I could use for this hypothesis, none of them seemed to fit so I asked chatgpt which would be best for this hypothesis. It suggested using the Spearman test which was briefly mentioned in one video in our learning content, but I couldn't find a demonstration of it so the following code was taken directly from chatgpt.

In [14]:
from scipy.stats import spearmanr
# Compute Spearman correlation
corr, p_value = spearmanr(df["Discount"], df["Profit"])

# Print result
print(f"🔹 Spearman Correlation (rho): {corr:.2f}")
print(f"🔹 p-value: {p_value:.4f}")

🔹 Spearman Correlation (rho): -0.54
🔹 p-value: 0.0000


The results of this tet show that there is a moderate to strong negative correlation between discount rate and profits through the rho score of -0.54 (a score of -1 would be a perfect negative correlation, 0 would mean no correlation and 1 would mean a perfect positive correlation).
The p-value is <0.05 meaning the result is highly significant and therefore not due to random chance.

As a result of this statistical test as well as by looking at the visuals portrayed by the graphs, we can confirm that higher discount rates do tend to lead to reduced profits in general, so we reject the null hypothesis in this case.

# Section 2

Hypothesis: Certain product categories are more profitable than others.

Null hypothesis: Each product category is equally profitable.

In [ ]:
# Grouping data by their Category and Sub-Category, summing Profit
category_profit = df.groupby(["Category", "Sub-Category"])["Profit"].sum().reset_index()

# Bar chart showing Profit by category
fig = px.bar(category_profit, x="Category", y="Profit", color="Sub-Category",
             title="Profit by Category & Sub-Category",
             labels={"Profit": "Total Profit ($)", "Category": "Product Category"},
             barmode="group") # was suggested by copilot to use barmode='group' to show the sub-categories side by side instead of stacked, looks better
fig.show()

This clearly shows a difference in the profit levels by category and sub category, will certain furniture sub categories even recording a loss. 

In [20]:
# Boxplot showing the distribution of profit by category
px.box(df, x='Category', y='Profit',
       title='Profit distribution by Category',
       labels={'Profit': 'Profit($)', 'Category': 'Category'})

The boxplot does not show as clear a difference between the main categories themselves, although there are clear differences in the distribution of profits between the 3. I will now do statistical tests to confirm whether the hypothesis is correct.

In [21]:
# Checking if Profit is normally distrubuted, will then decide whether to use ANOVA or Kruskal-Wallis test
import pingouin as pg

pg.normality(data=df['Profit'], alpha=0.05)

c:\Users\conor\Desktop\DA course\SuperstoreSales\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning:

scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 9994.



,W,pval,normal
Profit,0.633555,3.645313e-90,False


In [22]:
# As data is not normally distributed, I will use the Kruskal-Wallis test
pg.kruskal(data=df, dv='Profit', between='Category')

,Source,ddof1,H,p-unc
Kruskal,Category,2,445.910571,1.485079e-97


Since p-unc is < 0.05 then at least one of the product categories is more profitable than others and we can reject the null hypothesis and say that certain product categories are more profitable than others.

# Section 3

Hypothesis: Shipping mode and region affect delivery time

Null hypothesis: Shipping mode and region have no effect on the delivery time

In [29]:
# grouping by region and average delivery time
delivery_by_region = df.groupby('Region')['Delivery Time'].mean().reset_index()
delivery_by_region

,Region,Delivery Time
0,Central,4.058115
1,East,3.908708
2,South,3.958025
3,West,3.929753


In [36]:
fig = px.box(df, x="Region", y="Delivery Time",
             title="Distribution of Delivery Days by Region",
             labels={"Delivery Time": "Delivery Time", "Region": "Region"},
             color="Region")  # Different colors for each region

fig.show()

Looking at the mean of delivery time, and the boxplot that includes median delivery time, it looks like there is actually very little, if any, difference in delivery times across different regions. 

In [37]:
# checking if delivery time is normal to see what statistical test to use
pg.normality(data=df['Delivery Time'], alpha=0.05)

c:\Users\conor\Desktop\DA course\SuperstoreSales\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning:

scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 9994.



,W,pval,normal
Delivery Time,0.943394,8.019552e-52,False


In [48]:
# using Kruskal-Wallis test as data is not normally distributed
pg.kruskal(data=df, dv='Delivery Time', between='Region')

,Source,ddof1,H,p-unc
Kruskal,Region,3,11.246563,0.010465


The Kruskal-Wallis tests shows there is a difference between delivery times with the p-unc value of 0.010465 being less than 0.05. However, because I saw earlier through the mean values and the boxplot that there really isn't any real world difference between the delivery times by region, I didn't want to reject the null hypothesis in this case, so I asked chatgpt what to do in this case. It said I could perform another statistical test to check the practical impact of the difference between the delivery times, which I have inserted below.

In [47]:
kruskal_result = pg.kruskal(dv="Delivery Time", between="Region", data=df)

# Extract H-statistic and p-value correctly
H_statistic = kruskal_result["H"].values[0]  # Kruskal-Wallis H-statistic
p_value = kruskal_result["p-unc"].values[0]  # P-value

print(f"Kruskal-Wallis H-statistic: {H_statistic:.4f}")
print(f"P-value: {p_value:.4f}")
eta_squared = (H_statistic - (df["Region"].nunique() - 1)) / (len(df) - 1)
print(f"Effect Size (Eta-Squared): {eta_squared:.4f}")

Kruskal-Wallis H-statistic: 11.2466
P-value: 0.0105
Effect Size (Eta-Squared): 0.0008


With the Eta-squared value of 0.0008, this means that the difference between delivery times in each region is so small that it has no real world impact. We can accept the null hypothesis in this case when speaking about region specifically, but I will have to make more graphs and do more statistical tests to see if this is the case for the shipping mode as well.

In [50]:
fig = px.box(df, x='Ship Mode', y='Delivery Time',
             title='Delivery Time by Ship Mode',
             labels={'Delivery Time': 'Delivery Time', 'Ship Mode': 'Ship Mode'})
fig.show()

We can see from the boxplots in this case that there is a clear difference in delivery time when different shipping modes are selected. I will still do a statistical test to prove this mathematically.

In [51]:
# since we already know delivery time is not delivery distributed, we will again use the Kruskal-Wallis test
pg.kruskal(data=df, dv='Delivery Time', between='Ship Mode')

,Source,ddof1,H,p-unc
Kruskal,Ship Mode,3,6061.476173,0.0


From the Kruskal-Wallis test, we have a p-unc value of 0.0, which means there is a difference in delivery times by shipping mode. In this case, as the box plot also clearly shows this, I am happy to accept the hypothesis that shipping mode does have an effect on the delivery time.

In conclusion, I can't fully dismiss the null hypothesis on the basis that there is no real world difference in the delivery time based on what region the item is being delivered to, although I can dismiss the part that says there is no difference in delivery time based on shipping mode. 

# Section 4

Hypothesis: Larger orders leader to higher total profits.

Null hypothesis: There is no relationship between large orders and total profit.

In [58]:
# Going to do a scatter plot showing Quantity vs profit
fig = px.scatter(df, x="Sales", y="Profit",
                 title="Relationship Between Quantity and Profit",
                 labels={"Sales": "Total Spent on Order", "Profit": "Total Profit ($)"},
                 trendline="ols") 

fig.show()


From the scatter plot that includes a line of best fit, we can see that there is a trend upwards in regards to profit when the total amount spent on an order increases. The individual points on the chart are all over the place, so I will do some statistical tests to confirm the relationship.

In [59]:
# Checking if Sales and Profit are normally distributed
pg.normality(data=df[["Sales", "Profit"]], alpha=0.05)

c:\Users\conor\Desktop\DA course\SuperstoreSales\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning:

scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 9994.



,W,pval,normal
Sales,0.541230,1.656960e-95,False
Profit,0.633555,3.645313e-90,False


In [61]:
# Using the Spearman correlation test as the data is not normally distributed
corr, p_value = spearmanr(df["Sales"], df["Profit"])

print(f"🔹 Spearman Correlation: {corr:.2f}")
print(f"🔹 p-value: {p_value:.4f}")

🔹 Spearman Correlation: 0.52
🔹 p-value: 0.0000


Using the same test from Section 1, I can conclude that there is a moderate positive correlation between higher sales figures and higher levels of profit through the Spearman Correlation value of 0.52, and through the p-value of 0, I can confirm that this is not due to random chance.

As a result, I would have to reject the null hypothesis and conclude that larger orders(orders with larger $ amounts, not quantity of items) do lead to higher total profits.